In [ ]:
from Crypto.Cipher import AES
import os

KEY = "blueblurblueblur"
FLAG = "crypto{flag_is_here}"

# @chal.route('/ecbcbcwtf/decrypt/<ciphertext>/')
def decrypt(ciphertext):
    ciphertext = bytes.fromhex(ciphertext)

    cipher = AES.new(KEY, AES.MODE_ECB)
    try:
        decrypted = cipher.decrypt(ciphertext)
    except ValueError as e:
        return {"error": str(e)}

    return {"plaintext": decrypted.hex()}


# @chal.route('/ecbcbcwtf/encrypt_flag/')
def encrypt_flag():
    iv = os.urandom(16)

    cipher = AES.new(KEY, AES.MODE_CBC, iv)
    encrypted = cipher.encrypt(FLAG.encode())
    ciphertext = iv.hex() + encrypted.hex()

    return {"ciphertext": ciphertext}

In [4]:
from Crypto.Cipher import AES
import os
import requests

ENCODE_URL = "https://aes.cryptohack.org/ecbcbcwtf/encrypt_flag/"
DECODE_URL = "https://aes.cryptohack.org/ecbcbcwtf/decrypt/"

def get_respone(url):
    respone = requests.get(url).json()
    keys = respone.keys()
    for k in ("ciphertext", "plaintext"):
        if k in keys:
            return respone[k]

# Lấy encode:
ciphertext = get_respone(ENCODE_URL)

iv = ciphertext[:32]
cipher_blocks = [ciphertext[i:i+32] for i in range(32, len(ciphertext), 32)]

flag = ""
temp = ""

for block in cipher_blocks:
    plaintext = get_respone(DECODE_URL + block + "/")
    if len(iv) == len(plaintext):
        text = bytes(a ^ b for a, b in zip(bytes.fromhex(plaintext), bytes.fromhex(iv)))
        flag += text.hex()
    iv = block
    
flag = bytes.fromhex(flag).decode('utf-8')
print(flag)



crypto{3cb_5uck5_4v01d_17_!!!!!}
